# 구간 소요시간 데이터셋 품질 검사

In [13]:
import matplotlib.font_manager as fm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# 설치된 폰트 출력
font_list = [font.name for font in fm.fontManager.ttflist]
plt.rcParams['font.family'] = 'Malgun Gothic'

csv = '../../dataset/inference/route/241007_모든노선_8.1~8.14_평일_특성추가/inf.csv'
# csv = '../../dataset/train/route/241007_모든노선_8.1~8.14_평일_특성추가/소통_train.csv'


# TIME_GAP 이상치 PLOTLY


In [2]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import seaborn as sns

def show_time_gap_outliers_plotly(df):
    # df = df[df['TIME_GAP'] < 500000]
    df['TIME_GAP_MINUTES'] = df['TIME_GAP'] / 60
    
    Q1 = df['TIME_GAP'].quantile(0.25)
    Q3 = df['TIME_GAP'].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    # upper_bound = 300  # 상위 이상치 기준을 300초로 수동 설정

    lower_outliers = df[df['TIME_GAP'] < lower_bound]
    upper_outliers = df[df['TIME_GAP'] > upper_bound]

    outliers = df[(df['TIME_GAP'] < lower_bound) | (df['TIME_GAP'] > upper_bound)]
    print(f"이상치 개수: {len(outliers)}")
    print("\n이상치 상위 20개:")
    print(outliers.sort_values('TIME_GAP', ascending=False).head(20))

    print(f"상하위 이상치 기준점:")
    print(f" - Lower Bound (하위 이상치 기준점): {lower_bound:.2f}")
    print(f" - Upper Bound (상위 이상치 기준점): {upper_bound:.2f}")

    print(f"\n이상치 개수:")
    print(f" - 상위 이상치 개수 (Upper Outliers): {len(upper_outliers)}")
    print(f" - 하위 이상치 개수 (Lower Outliers): {len(lower_outliers)}")

    print("\n하위 이상치 데이터 (상위 5개):")
    print(lower_outliers.sort_values('TIME_GAP', ascending=True).head())

    # Plotly Box Plot 생성
    fig = go.Figure(
        data=[
            go.Box(y=df['TIME_GAP'], name="TIME_GAP", boxpoints='outliers') 
        ]
    )

    fig.update_layout(
        title='TIME_GAP 분포와 이상치 (Plotly)',
        yaxis_title='TIME_GAP (초)'
    )

    fig.show()
    
    # Plotly를 사용한 히스토그램 생성
    fig = go.Figure(data=[go.Histogram(x=df['TIME_GAP_MINUTES'],
                                       nbinsx=50,  # 막대의 개수
                                       autobinx=False,  # 자동 bin 설정 해제
                                       xbins=dict(start=0, end=50, size=1),  # 0~50분, 1분 간격
                                       marker_color='lightblue',
                                       opacity=0.75)])

    # 그래프 레이아웃 설정
    fig.update_layout(
        title='Distribution of TIME_GAP (Minutes)',
        xaxis_title='Time Gap (minutes)',
        yaxis_title='Frequency',
        bargap=0.1,  # 막대 사이의 간격
    )

    # x축 범위 설정 (0~50분)
    fig.update_xaxes(range=[0, 50])

    # 그래프 표시
    fig.show()
    
def visualize_time_gap_distribution(df):
    # TIME_GAP을 분 단위로 변환
    df['TIME_GAP_MINUTES'] = df['TIME_GAP'] / 60

    # 그래프 스타일 설정
    plt.figure(figsize=(12, 6))
    sns.set_style("whitegrid")

    # 전체 분포 시각화
    plt.subplot(1, 2, 1)
    sns.histplot(df['TIME_GAP_MINUTES'], kde=True, color='skyblue')
    plt.title('Distribution of TIME_GAP (Minutes)')
    plt.xlabel('Time Gap (minutes)')
    plt.ylabel('Frequency')

    # 0-50분 구간 상세 시각화
    plt.subplot(1, 2, 2)
    sns.histplot(df['TIME_GAP_MINUTES'][df['TIME_GAP_MINUTES'] <= 50], 
                 kde=True, color='lightgreen', bins=50)
    plt.title('Distribution of TIME_GAP (0-50 Minutes)')
    plt.xlabel('Time Gap (minutes)')
    plt.ylabel('Frequency')
    plt.xlim(0, 50)

    plt.tight_layout()
    plt.show()

    # 기술 통계량 출력
    print(df['TIME_GAP_MINUTES'].describe())

In [ ]:
# 추론용 데이터 로드
# csv = '../../dataset/train/travel/240924/수요일/train_combined.csv'
dtype_spec = {
    'DAY_TYPE': 'int8',
    # 'BUSROUTE_ID': 'str',
    'BUSINFOUNIT_ID': 'str',
    'LEN': 'int32',
    'DEP_TIME': 'str',
    'TIME_GAP': 'int32',  # int32는 NaN 값을 처리할 수 없으므로 float32로 변경
    'SPEED': 'int32',
    # 'DATA_TYPE': 'str'
}
# usecols = ['DAY_TYPE', 'BUSINFOUNIT_ID', 'LEN', 'DEP_TIME', 'SPEED']
# usecols = ['DAY_TYPE', 'BUSROUTE_ID', 'BUSINFOUNIT_ID', 'LEN', 'DEP_TIME', 'SPEED', 'TIME_GAP']
# usecols = ['DAY_TYPE', 'BUSROUTE_ID', 'BUSINFOUNIT_ID', 'LEN', 'DEP_TIME', 'SPEED', 'TIME_GAP', 'DATA_TYPE']
usecols = ['DAY_TYPE', 'BUSINFOUNIT_ID', 'LEN', 'DEP_TIME', 'TIME_GAP', 'SPEED']

data_pd = pd.read_csv(csv, skipinitialspace=True, usecols=usecols, dtype=dtype_spec)

# SPEED 컬럼의 NaN 또는 빈 값 제거
data_pd = data_pd.dropna(subset=['SPEED'])
data_pd = data_pd[data_pd['SPEED'] != '']
# TIME_GAP 컬럼의 NaN 또는 빈 값 제거
data_pd = data_pd.dropna(subset=['TIME_GAP'])
data_pd = data_pd[data_pd['TIME_GAP'] != '']

# data_pd.reset_index(drop=True, inplace=True)

# TIME_GAP 컬럼 정수로
data_pd['TIME_GAP'] = data_pd['TIME_GAP'].astype('int32')
# SPEED 컬럼 정수로 반올림
# data_pd['SPEED'] = data_pd['SPEED'].round().astype('int32')


# 1. DEP_TIME을 시:분 포맷으로 변환 (초 제거)
data_pd['DEP_TIME'] = pd.to_datetime(data_pd['DEP_TIME'], format='%H:%M').dt.strftime('%H:%M')
# data_pd['DEP_TIME'] = pd.to_datetime(data_pd['DEP_TIME'], format='%Y-%m-%d %H:%M:%S').dt.strftime('%H:%M')

# 2. 필요한 시간대 필터링 (07:00~09:00, 13:00~15:00, 18:00~20:00)
# valid_times = (
#     ((data_pd['DEP_TIME'] >= '07:00') & (data_pd['DEP_TIME'] <= '09:00')) |
#     ((data_pd['DEP_TIME'] >= '13:00') & (data_pd['DEP_TIME'] <= '15:00')) |
#     ((data_pd['DEP_TIME'] >= '18:00') & (data_pd['DEP_TIME'] <= '20:00'))
# )

# 3. 해당 시간대의 데이터만 추출
# data_pd = data_pd[valid_times]

show_time_gap_outliers_plotly(data_pd)
# visualize_time_gap_distribution(data_pd)

## 속도 이상치 PLOTLY

In [6]:
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px

def detect_speed_outliers(df):
    df['SPEED'] = (df['LEN'] / df['TIME_GAP']) * 3.6

    Q1 = df['SPEED'].quantile(0.25)
    Q3 = df['SPEED'].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    # upper_bound = 10

    lower_outliers = df[df['SPEED'] < lower_bound]
    upper_outliers = df[df['SPEED'] > upper_bound]
    outliers = pd.concat([lower_outliers, upper_outliers])

    print(f"이상치 개수: {len(outliers)}")
    print(f" - 상위 이상치 기준 (Upper Bound): {upper_bound:.2f} km/h")
    print(f" - 하위 이상치 기준 (Lower Bound): {lower_bound:.2f} km/h")

    print("\n상위 이상치 데이터 (상위 100개):")
    print(upper_outliers.sort_values('SPEED', ascending=False).head(100))

    print("\n하위 이상치 데이터 (상위 5개):")
    print(lower_outliers.sort_values('SPEED', ascending=True).head())

    # 박스플롯 시각화
    fig_box = go.Figure()
    fig_box.add_trace(go.Box(y=df['SPEED'], name='Speed'))
    fig_box.update_layout(
        title='Speed (km/h) Boxplot with Outliers',
        yaxis_title='Speed (km/h)',
        showlegend=False
    )
    fig_box.show()

    # 히스토그램 시각화
    fig_hist = go.Figure()
    fig_hist.add_trace(go.Histogram(x=df['SPEED'], nbinsx=30, name='Speed'))
    fig_hist.add_vline(x=lower_bound, line_dash="dash", line_color="red",
                       annotation_text=f"Lower Bound: {lower_bound:.2f} km/h",
                       annotation_position="top right")
    fig_hist.add_vline(x=upper_bound, line_dash="dash", line_color="red",
                       annotation_text=f"Upper Bound: {upper_bound:.2f} km/h",
                       annotation_position="top left")
    fig_hist.update_layout(
        title='Speed (km/h) Histogram with Outlier Boundaries',
        xaxis_title='Speed (km/h)',
        yaxis_title='Frequency',
        xaxis_range=[0, 200],
        # xaxis=dict(
        #     tickmode='linear',
        #     tick0=0,
        #     dtick=10
        # )
    )
    fig_hist.show()

    return outliers

In [ ]:
# 추론용 데이터 로드
# csv = '../../dataset/inference/travel/240920/inf_combined.csv'
# csv = '../../dataset/train/travel/240920/금요일/소통시간_4-5금요일학습데이터.csv'
dtype_spec = {
    'DAY_TYPE': 'int8',
    # 'BUSROUTE_ID': 'str',
    'BUSINFOUNIT_ID': 'str',
    'LEN': 'int32',
    'DEP_TIME': 'str',
    'TIME_GAP': 'int32',  # int32는 NaN 값을 처리할 수 없으므로 float32로 변경
    'SPEED': 'int32',
    # 'DATA_TYPE': 'str'
}
usecols = ['DAY_TYPE', 'BUSINFOUNIT_ID', 'LEN', 'DEP_TIME', 'TIME_GAP', 'SPEED']
# usecols = ['DAY_TYPE', 'BUSROUTE_ID', 'BUSINFOUNIT_ID', 'LEN', 'DEP_TIME', 'SPEED', 'TIME_GAP', 'DATA_TYPE']

data_pd = pd.read_csv(csv, skipinitialspace=True, usecols=usecols, dtype=dtype_spec)

# SPEED 컬럼의 NaN 또는 빈 값 제거
data_pd = data_pd.dropna(subset=['SPEED'])
data_pd = data_pd[data_pd['SPEED'] != '']
# TIME_GAP 컬럼의 NaN 또는 빈 값 제거
data_pd = data_pd.dropna(subset=['TIME_GAP'])
data_pd = data_pd[data_pd['TIME_GAP'] != '']

# data_pd.reset_index(drop=True, inplace=True)

# TIME_GAP 컬럼 정수로
data_pd['TIME_GAP'] = data_pd['TIME_GAP'].astype('int32')
# SPEED 컬럼 정수로 반올림
data_pd['SPEED'] = data_pd['SPEED'].round().astype('int32')


# 1. DEP_TIME을 시:분 포맷으로 변환 (초 제거)
data_pd['DEP_TIME'] = pd.to_datetime(data_pd['DEP_TIME'], format='%H:%M').dt.strftime('%H:%M')
# data_pd['DEP_TIME'] = pd.to_datetime(data_pd['DEP_TIME'], format='%Y-%m-%d %H:%M:%S').dt.strftime('%H:%M')

# 2. 필요한 시간대 필터링 (07:00~09:00, 13:00~15:00, 18:00~20:00)
# valid_times = (
#     ((data_pd['DEP_TIME'] >= '07:00') & (data_pd['DEP_TIME'] <= '09:00')) |
#     ((data_pd['DEP_TIME'] >= '13:00') & (data_pd['DEP_TIME'] <= '15:00')) |
#     ((data_pd['DEP_TIME'] >= '18:00') & (data_pd['DEP_TIME'] <= '20:00'))
# )

# 3. 해당 시간대의 데이터만 추출
# data_pd = data_pd[valid_times]

detect_speed_outliers(data_pd)

# 필터링 파일 저장

In [16]:
import pandas as pd

def filter_and_save_data(df, output_file='filtered_data.csv'):
    # 1. LEN이 0인 행 제외
    original_count = len(df)
    filtered_df = df[df['LEN'] > 0]
    removed_count_len = original_count - len(filtered_df)
    print(f"구간 길이(LEN)가 0인 {removed_count_len}개의 행이 제거되었습니다.")
    
    # 2. 중복 행 제거
    original_count = len(filtered_df)
    filtered_df = filtered_df.drop_duplicates()
    removed_count_duplicates = original_count - len(filtered_df)
    print(f"중복된 {removed_count_duplicates}개의 행이 제거되었습니다.")
    
    # 3. TIME_GAP이 0이거나 300보다 큰 행 제외
    original_count = len(filtered_df)
    filtered_df = filtered_df[(filtered_df['TIME_GAP'] > 5) & (filtered_df['TIME_GAP'] <= 300)]
    removed_count_time_gap = original_count - len(filtered_df)
    print(f"소요시간(TIME_GAP)이 5미만 이거나 300초보다 큰 {removed_count_time_gap}개의 행이 제거되었습니다.")
    
    # # 4. 시속 5km/h 이하 또는 90km/h 초과인 행 제외
    # filtered_df['SPEED'] = ((filtered_df['LEN'] / filtered_df['TIME_GAP']) * 3.6).round()
    original_count = len(filtered_df)
    filtered_df = filtered_df[(filtered_df['SPEED'] > 5) & (filtered_df['SPEED'] <= 55)]
    removed_count_speed = original_count - len(filtered_df)
    print(f"시속 5km/h 이하 또는 55km/h 초과인 {removed_count_speed}개의 행이 제거되었습니다.")
    
    original_count = len(filtered_df)
    filtered_df = filtered_df[(filtered_df['GPS_COORDX'] != 0) & (filtered_df['GPS_COORDY'] != 0)] # 0이 아닌 행만 남김
    removed_count_coord = original_count - len(filtered_df)
    print(f"GPS 좌표가 0인 {removed_count_coord}개의 행이 제거되었습니다.")
    
    # 4. 초 단위 있으면 없앰
    # filtered_df['DEP_TIME'] = pd.to_datetime(filtered_df['DEP_TIME'], format='%Y-%m-%d %H:%M:%S').dt.strftime('%H:%M')
    
    valid_times = (
        ((filtered_df['DEP_TIME'] >= '07:00') & (filtered_df['DEP_TIME'] <= '09:00')) |
        ((filtered_df['DEP_TIME'] >= '13:00') & (filtered_df['DEP_TIME'] <= '15:00')) |
        ((filtered_df['DEP_TIME'] >= '18:00') & (filtered_df['DEP_TIME'] <= '20:00'))
    )

    # 3. 해당 시간대의 데이터만 추출
    filtered_df = filtered_df[valid_times]
    
    # 중복 제거
    filtered_df = filtered_df.drop_duplicates(subset=['BUSROUTE_ID', 'BUSINFOUNIT_ID', 'DEP_TIME'], keep='first')
    
    # 결과를 새로운 CSV 파일로 저장
    filtered_df.to_csv(output_file, index=False)
    print(f"필터링된 데이터를 '{output_file}'로 저장했습니다.")

In [17]:
import os

# csv = '../../dataset/inference/travel/240920/inf_combined.csv'
# csv = '../../dataset/train/travel/240920/금요일/소통시간_4-5금요일학습데이터.csv'

dtype_spec = {
    'DAY_TYPE': 'int8',
    'BUSROUTE_ID': 'str',
    'PEEK_ALLOC': 'int16',
    'NPEEK_ALLOC': 'int16',
    'ROUTE_LEN': 'int32',
    'BUSSTOP_CNT': 'int16',
    'BUSINFOUNIT_ID': 'str',
    'INFOUNIT_SEQ': 'int16',
    'LEN': 'int32',
    'GPS_COORDX': 'float32',
    'GPS_COORDY': 'float32',
    'COLLECT_DATE': 'str',
    'DEP_TIME': 'str',
    'TIME_GAP': 'float32',  # int32는 NaN 값을 처리할 수 없으므로 float32로 변경
    'SPEED': 'float32'
}
# usecols = ['DAY_TYPE', 'BUSROUTE_ID', 'BUSINFOUNIT_ID', 'LEN', 'DEP_TIME', 'SPEED', 'TIME_GAP']
usecols = ['DAY_TYPE', 'BUSROUTE_ID', 'PEEK_ALLOC', 'NPEEK_ALLOC', 'ROUTE_LEN', 'BUSSTOP_CNT', 'BUSINFOUNIT_ID', 'INFOUNIT_SEQ', 'LEN', 'GPS_COORDX', 'GPS_COORDY', 'COLLECT_DATE', 'DEP_TIME', 'SPEED', 'TIME_GAP']
# usecols = ['DAY_TYPE', 'BUSROUTE_ID', 'BUSINFOUNIT_ID', 'LEN', 'COLLECT_DATE', 'DEP_TIME', 'SPEED', 'TIME_GAP']
# usecols = ['DAY_TYPE', 'BUSROUTE_ID', 'BUSINFOUNIT_ID', 'LEN', 'DEP_TIME', 'SPEED', 'TIME_GAP']
# usecols = ['DAY_TYPE', 'BUSROUTE_ID', 'BUSINFOUNIT_ID', 'LEN', 'DEP_TIME', 'TIME_GAP']

data_pd = pd.read_csv(csv, skipinitialspace=True, usecols=usecols, dtype=dtype_spec)

# TIME_GAP 컬럼 정수로
data_pd['TIME_GAP'] = data_pd['TIME_GAP'].astype('int32')
# SPEED 컬럼 정수로 반올림
data_pd['SPEED'] = data_pd['SPEED'].round().astype('int32')

# 결과 저장
input_filename = os.path.basename(csv)
output_filename = os.path.splitext(input_filename)[0] + '_filtered.csv'
result_file = os.path.join(os.path.dirname(csv), output_filename)

filter_and_save_data(data_pd, result_file)

구간 길이(LEN)가 0인 0개의 행이 제거되었습니다.
중복된 9305개의 행이 제거되었습니다.
소요시간(TIME_GAP)이 5미만 이거나 300초보다 큰 1383개의 행이 제거되었습니다.
시속 5km/h 이하 또는 55km/h 초과인 3148개의 행이 제거되었습니다.
GPS 좌표가 0인 4500개의 행이 제거되었습니다.
필터링된 데이터를 '../../dataset/inference/route/241007_모든노선_8.1~8.14_평일_특성추가\inf_filtered.csv'로 저장했습니다.
